**Train_Valid_Test**

In [1]:
import pandas as pd
!pip install ijson
import ijson


     |████████████████████████████████| 126 kB 13.3 MB/s 


In [ ]:

from os import sep
import pandas as pd

templates_df = pd.read_csv('/content/drive/MyDrive/NLP/ans.tsv', sep="\t")

classes = {}

for idx, item in templates_df.iterrows():
    classes[item['Template']] = idx


def assignClassNumber(template):
    return classes[template]


templates_df['Class'] = templates_df['Template'].apply(assignClassNumber)

templates_df

templates_df.to_csv('/content/drive/MyDrive/NLP/TemplatePool.csv', index=False)

templates_df


In [9]:
!pip install ijson
import pandas as pd
import ijson



templatePool = pd.read_csv('TemplatePool.csv')
templatePool

limit = 0
data = []
with open('/content/drive/MyDrive/NLP/biencoder-trivia-train.json', 'r') as f:
    for item in ijson.items(f, "item"):
        if limit == 120:
            break
        data.append(item)
        limit += 1


data[0]['positive_ctxs']

test = list()
for item in data:
    if len(item['positive_ctxs']) > 0:
        for ctx in item['positive_ctxs']:
            test.append(
                {
                    "context": ctx['text'],
                    "template": templatePool[templatePool['Question'] == item['question']]['Template'].values[0],
                    "question": templatePool[templatePool['Question'] == item['question']]['Question'].values[0]
                }
            )

df = pd.DataFrame(test)

df.to_csv('/content/drive/MyDrive/NLP/train_seq.csv', index=False)

len(df['context'].max().split())



In [ ]:
train_seq = pd.read_csv('/content/drive/MyDrive/NLP/train_seq.csv')
df_1 = train_seq.iloc[:500,:]
df_2 = train_seq.iloc[501:600,:]
df_3 = train_seq.iloc[601:,:]

# Seq2Seq


In [2]:
#!pip install torch==1.4.0
!pip install transformers==2.9.0
!pip install pytorch_lightning==0.7.5

     |████████████████████████████████| 635 kB 12.5 MB/s 
     |████████████████████████████████| 1.2 MB 40.9 MB/s 
     |████████████████████████████████| 5.6 MB 24.3 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
     |████████████████████████████████| 233 kB 12.1 MB/s 
     |████████████████████████████████| 829 kB 50.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=982a13cae76812678f09a6750253b94b493bfc1025ea030d74204b97e0e03e25
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [7]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)


class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams

        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)

    def is_logger(self):
        return self.trainer.proc_rank <= 0

    def forward(
            self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            lm_labels=lm_labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            lm_labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step()
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train5", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True,
                                num_workers=4)
        t_total = (
                (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="valid5", args=self.hparams)
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:

logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [9]:

args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=6,
    eval_batch_size=6,
    num_train_epochs=4,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)


In [10]:
train_path = "/content/drive/MyDrive/NLP/train5.csv"
val_path = "/content/drive/MyDrive/NLP/valid5.csv"

train = pd.read_csv(train_path)
print (train.head())


                                             context  ...                                           question
0  Sinclair Lewis Harry Sinclair Lewis (February ...  ...  Which American-born Sinclair won the Nobel Pri...
1  Sinclair Lewis Boyhood Home The Sinclair Lewis...  ...  Which American-born Sinclair won the Nobel Pri...
2  of an immense exuberance, organic in its form,...  ...  Which American-born Sinclair won the Nobel Pri...
3  regular contact with the theatre. Her father, ...  ...         Where in England was Dame Judi Dench born?
4  Jeffery Dench Jeffery Danny Dench (29 April 19...  ...         Where in England was Dame Judi Dench born?

[5 rows x 3 columns]


In [11]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

DEBUG:filelock:Attempting to acquire lock 139785102145488 on /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f.lock
DEBUG:filelock:Lock 139785102145488 acquired on /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp03zmd6wu


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model in cache at /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f
DEBUG:filelock:Attempting to release lock 139785102145488 on /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f.lock
DEBUG:filelock:Lock 139785102145488 released on /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f.lock
INFO:transformers.tokenization_utils:loading file https://s3

In [12]:
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, max_len=256):
        self.path = os.path.join(data_dir, type_path + '.csv')

        self.passage_column = "context"
        self.true_false = "template"
        self.target_column = "question"
        self.data = pd.read_csv(self.path)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            passage,true_false,target = self.data.loc[idx, self.passage_column],self.data.loc[idx, self.true_false], self.data.loc[idx, self.target_column]
            true_false = str(true_false)
            
            # input_ = "paraphrase: "+ input_ + ' </s>'
            input_ = "truefalse: %s passage: %s </s>" % (true_false,passage)
            target = target + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)


In [13]:
dataset = ParaphraseDataset(tokenizer, '/content/drive/MyDrive/NLP/train5', '/content/drive/MyDrive/NLP/valid5', 256)
print("Val dataset: ",len(dataset))

data = dataset[2]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))

if not os.path.exists('t5_boolq'):
    os.makedirs('t5_boolq')

args_dict.update({'data_dir': '/content/drive/MyDrive/NLP', 'output_dir': 't5_boolq', 'num_train_epochs':4,'max_seq_length':256})
args = argparse.Namespace(**args_dict)
print(args_dict)

Val dataset:  165
truefalse: In what _ were _ ground troops first dispatched to _? passage: on. As public support decreased, opposition grew. The Gallup News Service began asking the American public whether it was a "mistake to send troops to Vietnam" in August 1965. At the time less than a quarter of Americans polled, 24%, believed it was a mistake to send troops to Vietnam while 60% of Americans polled believed the opposite. Three years later, in September 1968, 54% of Americans polled believed it was a mistake to send troops to Vietnam while 37% believed it was not a mistake. A 1965 Gallup Poll asked the question, "Have you ever felt the urge to
In what year were US ground troops first dispatched to Vietnam?
{'data_dir': '/content/drive/MyDrive/NLP', 'output_dir': 't5_boolq', 'model_name_or_path': 't5-base', 'tokenizer_name_or_path': 't5-base', 'max_seq_length': 256, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 6, 'eval_

In [14]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=1
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

def get_dataset(tokenizer, type_path, args):
  return ParaphraseDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)



print ("Initialize model")
model = T5FineTuner(args)

trainer = pl.Trainer(**train_params)

print (" Training model")
trainer.fit(model)

print ("training finished")

print ("Saving model")
model.model.save_pretrained('/content/drive/MyDrive/NLP/t5_boolq')

print ("Saved model")

Initialize model


DEBUG:filelock:Attempting to acquire lock 139785057432976 on /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b.lock
DEBUG:filelock:Lock 139785057432976 acquired on /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp4e0f6_x1


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json in cache at /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b
DEBUG:filelock:Attempting to release lock 139785057432976 on /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b.lock
DEBUG:filelock:Lock 139785057432976 released on /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b.lock
INFO:transformers.configuration_utils:loading configurat

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

INFO:__main__:***** Validation results *****
INFO:__main__:avg_train_loss = tensor(0.2764, device='cuda:0')

INFO:__main__:avg_val_loss = tensor(1.3087, device='cuda:0')

INFO:__main__:epoch = 2

INFO:__main__:loss = tensor(0.2808, device='cuda:0')

INFO:__main__:train_loss = tensor(0.2808, device='cuda:0')

INFO:__main__:val_loss = tensor(1.3087, device='cuda:0')



training finished
Saving model


INFO:transformers.configuration_utils:Configuration saved in /content/drive/MyDrive/NLP/t5_boolq/config.json
INFO:transformers.modeling_utils:Model weights saved in /content/drive/MyDrive/NLP/t5_boolq/pytorch_model.bin


Saved model


In [12]:
!cp "/content/t5_boolq" "/content/drive/MyDrive/NLP"

cp: cannot stat '/content/t5_boolq': No such file or directory


In [27]:

import time
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/NLP/t5_boolq')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print ("device ",device)
model = model.to(device)


def greedy_decoding (inp_ids,attn_mask):
  greedy_output = model.generate(input_ids=inp_ids, attention_mask=attn_mask, max_length=256)
  Question =  tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
  return Question.strip().capitalize()


def beam_search_decoding (inp_ids,attn_mask):
  beam_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               num_beams=10,
                               num_return_sequences=3,
                               no_repeat_ngram_size=2,
                               early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True) for out in
               beam_output]
  return [Question.strip().capitalize() for Question in Questions]


def topkp_decoding (inp_ids,attn_mask):
  topkp_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               do_sample=True,
                               top_k=40,
                               top_p=0.80,
                               num_return_sequences=3,
                                no_repeat_ngram_size=2,
                                early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True,clean_up_tokenization_spaces=True) for out in topkp_output]
  return [Question.strip().capitalize() for Question in Questions]

#start = time.time()


#passage = '''elephant. Fleegle would assume the role of leader of the Banana Splits, and preside at club meetings. The characters were played by actors in voluminous fleecy costumes, similar to later Sid and Marty Krofft characters such as "H.R. Pufnstuf". While Snorky "spoke" in honking noises, the other three spoke in English—Drooper with a Southern drawl in the manner of Michael Nesmith, and Fleegle with a pronounced lisp. The Splits' segments, including songs-of-the-week and comedy skits, served as wraparounds for a number of individual segments. In the second season, "The Three Musketeers" segments were replaced with repeats of "The Hillbilly Bears"'''
#truefalse = "In The Banana Splits what sort of animal was _ ?"

#passage='''Neil Armstrong Neil Alden Armstrong (August 5, 1930 – August 25, 2012) was an American astronaut and aeronautical engineer who was the first person to walk on the Moon. He was also a naval aviator, test pilot, and university professor. A graduate of Purdue University, Armstrong studied aeronautical engineering with his college tuition paid for by the U.S. Navy under the Holloway Plan. He became a midshipman in 1949, and a naval aviator the following year. He saw action in the Korean War, flying the Grumman F9F Panther from the aircraft carrier . In September 1951, he was hit by'''
#truefalse = "_ was a pilot in which war ?"

passage = '''Project 42 and 50 ASW vessels). On 1 April 1961, the division was renamed the 2nd Anti-Submarine Warfare Division. On 1 July 1958, the Northern Fleet raised the Soviet Navy ensign over the first Soviet nuclear submarine, K-3 Leninsky Komsomol. Following the 1958 voyage of USS "Nautilus", the "Leninsky Komsomol" (named for Vladimir Lenin’s Komsomol) traveled under the Arctic ice and surfaced at the North Pole on 17 July 1962. Russian submarines have visited the North Pole region more than 300 times since then. Two nuclear submarines of the Northern Fleet made a journey under the Arctic ice cap and'''
truefalse = "Which _ nuclear submarine was the _ to sail across _ under the ice cap ?"

text = "truefalse: %s passage: %s </s>" % (passage, truefalse)


max_len = 256

encoding = tokenizer.encode_plus(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)



print ("Context: ",passage)
# print ("\nGenerated Question: ",truefalse)

# output = greedy_decoding(input_ids,attention_masks)
# print ("\nGreedy decoding:: ",output)

output = beam_search_decoding(input_ids,attention_masks)
print ("\nBeam decoding [Most accurate questions] ::\n")
for out in output:
    print(out)


output = topkp_decoding(input_ids,attention_masks)
print ("\nTopKP decoding [Not very accurate but more variety in questions] ::\n")
for out in output:
    print (out)

end = time.time()
print ("\nTime elapsed ", end-start)
print ("\n")


INFO:transformers.configuration_utils:loading configuration file /content/drive/MyDrive/NLP/t5_boolq/config.json
INFO:transformers.configuration_utils:Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate E

Context:  Project 42 and 50 ASW vessels). On 1 April 1961, the division was renamed the 2nd Anti-Submarine Warfare Division. On 1 July 1958, the Northern Fleet raised the Soviet Navy ensign over the first Soviet nuclear submarine, K-3 Leninsky Komsomol. Following the 1958 voyage of USS "Nautilus", the "Leninsky Komsomol" (named for Vladimir Lenin’s Komsomol) traveled under the Arctic ice and surfaced at the North Pole on 17 July 1962. Russian submarines have visited the North Pole region more than 300 times since then. Two nuclear submarines of the Northern Fleet made a journey under the Arctic ice cap and

Beam decoding [Most accurate questions] ::

Which russian nuclear submarine was the first to sail across the arctic?
Which russian nuclear submarine was the first to sail across the arctic under the ice?
Which nuclear submarine was the first to sail across the arctic under the ice cap?

TopKP decoding [Not very accurate but more variety in questions] ::

Which nuclear submarine was 

In [31]:

col_names = ['Context','Template','T5-Question1','T5-Question2']
result = pd.DataFrame(columns=col_names)
test = pd.read_csv('/content/drive/MyDrive/NLP/test5.csv')
for i in range(90):
  passage = test.iat[i,0]
  truefalse = test.iat[i,1]
  text = "truefalse: %s passage: %s </s>" % (passage, truefalse)
  max_len = 256
  
  encoding = tokenizer.encode_plus(text, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

  #print ("Context: ",passage)
  # print ("\nGenerated Question: ",truefalse)

  # output = greedy_decoding(input_ids,attention_masks)
  # print ("\nGreedy decoding:: ",output)

  output1 = beam_search_decoding(input_ids,attention_masks)
  output2 = topkp_decoding(input_ids,attention_masks)
 
  new_row = {'Context':passage, 'Template':truefalse, 'T5-Question1':output1[2], 'T5-Question2':output2[2]}
  #append row to the dataframe
  result = result.append(new_row, ignore_index=True)

result.to_csv('/content/drive/MyDrive/NLP/seq2seq.tsv',sep='\t',index=False)